# Using *spotter* for splicing dependency analysis and exploring exon-drug associations

In [1]:
import os
import pandas as pd
from target_spotter import SplicingDependency, DrugAssociation

## Load data

In [2]:
# paths
ROOT = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(ROOT,"data","examples","CCLE")
splicing_file = os.path.join(DATA_DIR, "splicing_EX.tsv.gz")
genexpr_file = os.path.join(DATA_DIR, "genexpr.tsv.gz")

# read
splicing = pd.read_table(splicing_file).set_index("EVENT") # PSI
genexpr = pd.read_table(genexpr_file).set_index("ID") # TPM

In [3]:
splicing.head()

,ACH-000415,ACH-000894,ACH-000422,ACH-000358,ACH-000468,ACH-000502,ACH-000609,ACH-000636,ACH-000715,ACH-000653,...,ACH-000969,ACH-000277,ACH-000036,ACH-000197,ACH-000208,ACH-000359,ACH-000440,ACH-000804,ACH-000174,ACH-000934
EVENT,,,,,,,,,,,,,,,,,,,,,
HsaEX0067681,1.77,1.94,1.18,7.30,0.98,0.00,NaN,8.12,1.67,NaN,...,1.75,0.00,0.69,NaN,2.95,3.83,NaN,2.73,2.25,NaN
HsaEX6078702,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.0
HsaEX0056692,5.88,38.53,24.35,9.09,16.96,13.33,11.69,1.79,9.63,23.47,...,10.30,12.32,17.07,20.83,22.67,36.00,8.36,40.81,18.92,NaN
HsaEX0056690,88.89,93.85,83.78,87.65,91.11,94.20,93.33,83.87,78.12,95.04,...,86.14,91.26,95.83,NaN,91.30,95.35,94.31,95.40,95.83,NaN
HsaEX0056691,100.00,100.00,96.72,100.00,95.24,100.00,100.00,98.78,97.94,97.98,...,98.93,99.09,96.96,100.00,100.00,92.52,100.00,100.00,100.00,100.0


In [4]:
genexpr.head()

,ACH-000415,ACH-000894,ACH-000422,ACH-000358,ACH-000468,ACH-000502,ACH-000609,ACH-000636,ACH-000715,ACH-000653,...,ACH-000969,ACH-000277,ACH-000036,ACH-000197,ACH-000208,ACH-000359,ACH-000440,ACH-000804,ACH-000174,ACH-000934
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,4.389567,7.281791,5.064366,6.165912,3.939227,3.806324,0.584963,3.720278,3.510962,0.505891,...,4.479619,1.992768,5.307064,1.144046,4.950935,3.885574,0.632268,5.334497,5.068241,2.625270
ENSG00000000005,0.000000,0.014355,0.555816,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.028569,0.000000,0.000000,0.028569,0.000000
ENSG00000000419,5.147714,6.803744,6.841596,5.928607,6.625417,7.470537,6.926237,6.569248,6.887647,6.498730,...,7.334050,8.805421,7.083958,6.962318,6.462380,7.110509,6.588715,7.126291,7.679973,7.010444
ENSG00000000457,1.000000,2.469886,2.931683,2.726831,1.963474,2.555816,1.778209,3.303050,2.090853,3.195348,...,3.145677,3.533563,2.361768,3.553361,2.400538,2.124328,3.943921,3.001802,2.060047,2.833902
ENSG00000000460,1.555816,3.811471,3.834913,4.347666,3.228049,2.925999,3.169925,4.613532,2.589763,4.125155,...,3.270529,3.904002,1.978196,5.164706,3.646163,3.626439,3.816600,4.672425,3.625270,3.538538


## Predict drug sensitivity

### Compute splicing dependency

In [5]:
estimator = SplicingDependency()
spldep_means, max_harm_score_means = estimator.predict(splicing, genexpr)
spldep_means

Loading defaults...
Preprocessing inputs...
Standardizing data...
Computing splicing dependencies...


100%|██████████████████████████████████████| 1073/1073 [00:04<00:00, 266.78it/s]


,ACH-000889,ACH-000080,ACH-000173,ACH-000414,ACH-000208,ACH-000444,ACH-000430,ACH-000861,ACH-000468,ACH-000714,...,ACH-000842,ACH-000422,ACH-000555,ACH-000056,ACH-000784,ACH-000764,ACH-000969,ACH-000934,ACH-000957,ACH-000277
HsaEX6065058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.077509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HsaEX6065028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.166987,NaN,NaN,NaN
HsaEX6008208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.017849,NaN,NaN,NaN,NaN,NaN
HsaEX1001338,NaN,NaN,-0.076295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.053080,NaN,NaN,NaN,NaN,NaN,NaN
HsaEX0001886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.118040,0.094016,0.071596,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HsaEX0014912,0.007914,-0.098415,-0.051148,-0.033608,-0.011182,0.005148,0.047746,-0.001850,0.004546,-0.089423,...,-0.028651,-0.049158,-0.022326,-0.037989,-0.007192,-0.042378,-0.000205,0.045479,-0.007692,-0.043506
HsaEX0073989,-0.050063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.034737,NaN,-0.028214,-0.021731
HsaEX7109159,-0.250018,-0.328524,-0.446593,-0.280428,-0.365343,-0.300575,-0.290842,-0.323265,-0.304486,-0.323678,...,-0.390806,-0.404623,-0.420477,-0.293094,-0.291867,-0.252468,-0.324466,-0.440468,-0.414230,-0.436812
HsaEX7109160,-0.250472,-0.328869,-0.444978,-0.278123,-0.365418,-0.302806,-0.287013,-0.323519,-0.305841,-0.307735,...,-0.382253,-0.401836,-0.422000,-0.295069,-0.293640,-0.252709,-0.324961,-0.440873,-0.414439,-0.435643


### Compute drug sensitivity

In [6]:
estimator = DrugAssociation()
ic50_by_drug, ic50_by_exon = estimator.predict(spldep_means)

Loading defaults...
Preprocessing inputs...
Estimating drug responses...


In [7]:
ic50_by_drug

,ID,sample,predicted_ic50
0,1001_2000.0,ACH-000889,3.422479
1,1001_2000.0,ACH-000080,3.751697
2,1001_2000.0,ACH-000173,3.792793
3,1001_2000.0,ACH-000414,3.713157
4,1001_2000.0,ACH-000208,3.528714
...,...,...,...
45,9_1.0,ACH-000764,-0.165684
46,9_1.0,ACH-000969,0.008358
47,9_1.0,ACH-000934,-0.103984
48,9_1.0,ACH-000957,0.109201


In [8]:
ic50_by_exon

,ID,EVENT,ENSEMBL,GENE,ACH-000889,ACH-000080,ACH-000173,ACH-000414,ACH-000208,ACH-000430,...,ACH-000842,ACH-000422,ACH-000555,ACH-000056,ACH-000784,ACH-000764,ACH-000969,ACH-000934,ACH-000957,ACH-000277
0,1_2.0,HsaEX6065058,ENSG00000175899,A2M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.675744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_2.0,HsaEX6065028,ENSG00000166535,A2ML1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_2.0,HsaEX6008208,ENSG00000179869,ABCA13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.551692,NaN,NaN,NaN,NaN,NaN
3,1_2.0,HsaEX1001338,ENSG00000005471,ABCB4,NaN,NaN,2.436288,NaN,NaN,NaN,...,NaN,NaN,NaN,2.468079,NaN,NaN,NaN,NaN,NaN,NaN
4,1_2.0,HsaEX0001886,ENSG00000159640,ACE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.633315,2.604261,3.203030,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343355,1530_10.0,HsaEX0014912,ENSG00000153774,CFDP1,3.465803,3.377949,3.433243,3.442328,3.435875,3.629287,...,3.539898,3.401568,3.492935,3.473399,3.472927,3.448431,3.578291,3.613434,3.629757,3.598649
343356,1530_10.0,HsaEX0073989,ENSG00000161551,ZNF577,4.452614,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.250901,NaN,4.530149,4.525874
343357,1530_10.0,HsaEX7109159,ENSG00000239900,ADSL,3.587498,3.658223,3.639984,3.625915,3.584126,3.689747,...,3.713540,3.606247,3.656659,3.663214,3.616484,3.645722,3.710341,3.674440,3.771126,3.793511
343358,1530_10.0,HsaEX7109160,ENSG00000239900,ADSL,3.605473,3.682138,3.672978,3.646011,3.611030,3.710434,...,3.741657,3.635986,3.687776,3.684466,3.637723,3.663758,3.733845,3.707007,3.801452,3.825464
